<a href="https://colab.research.google.com/github/juhumkwon/source_code/blob/main/a_10_2__rnn_if_you_want_you.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import tensorflow as tf

# 1. 데이터 준비
text = "if you want you"  # 학습할 텍스트
chars = sorted(set(text))  # 고유한 문자들
char_to_idx = {char: idx for idx, char in enumerate(chars)}  # 문자 -> 인덱스 변환
idx_to_char = {idx: char for idx, char in enumerate(chars)}  # 인덱스 -> 문자 변환

# 데이터 전처리 (입력 시퀀스와 출력 시퀀스 생성)
input_seq = [char_to_idx[char] for char in text[:-1]]  # 마지막 문자는 제외
output_seq = [char_to_idx[char] for char in text[1:]]  # 첫 번째 문자는 제외

# 입력과 출력을 (1, 시퀀스 길이) 형태로 변환
input_seq = np.array(input_seq).reshape(1, -1)
output_seq = np.array(output_seq).reshape(1, -1)

vocab_size = len(chars)  # 고유한 문자 개수
embedding_dim = 8        # 임베딩 차원 수
input_length = input_seq.shape[1]  # 입력 시퀀스 길이

# 2. RNN 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=input_length),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])

# 모델 컴파일
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# 3. 학습
model.fit(input_seq, output_seq, epochs=100)

# 4. 예측
def generate_text(model, start_string, num_generate):
    # 입력 텍스트를 숫자로 변환
    input_eval = [char_to_idx[char] for char in start_string]
    input_eval = np.array(input_eval).reshape(1, -1)

    generated_text = start_string

    # 문자를 예측
    for _ in range(num_generate):
        predictions = model(input_eval)
        predicted_id = np.argmax(predictions[0, -1, :])

        # 예측한 문자를 기존 텍스트에 추가
        predicted_char = idx_to_char[predicted_id]
        generated_text += predicted_char
        print("generated_text=", generated_text)

        # 다음 입력 시퀀스를 생성 (문자 추가)
        input_eval = np.append(input_eval, [[predicted_id]], axis=1)

    return generated_text

# 5. 예시 텍스트 생성
print(generate_text(model, start_string="if you ", num_generate=10))

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 2.3074
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - loss: 2.2867
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 2.2661
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 2.2453
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 2.2236
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 2.2007
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 2.1760
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 2.1491
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 2.1195
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 2.0869
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 2.0511
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 2.0119
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 1.9694
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 1.9237
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 1.8750
Epoch 16/100
1/1 ━━━━━━━━━━━━━━━

In [ ]:
"""
predictions는 모델의 예측 결과로, 보통 (배치 크기, 시퀀스 길이, 어휘 크기) 형태의 3D 배열입니다.
이때 predictions[0, -1, :]는 predictions[0][4][:]와 동일하며,
다섯 번째 타임스텝(마지막)의 예측 확률 분포 [0.05, 0.15, 0.7, 0.1]을 가져옵니다.
이 분포는 다음 문자로 ['a', 'b', 'c', 'd'] 중 어느 것이 나올지에 대한 확률입니다:
'a': 5% 확률
'b': 15% 확률
'c': 70% 확률 (가장 높은 확률)
'd': 10% 확률
여기서 np.argmax(predictions[0, -1, :])를 사용하면 가장 높은 확률을 가진 인덱스인 2를 반환하게 됩니다.
이는 vocabulary에서 2번째 인덱스에 해당하는 문자, 즉 'c'가 다음에 나올 확률이 가장 높다고 모델이 예측했음을 의미합니다.
"""

import numpy as np
predictions = [
    [  # 배치 1
        [0.1, 0.3, 0.4, 0.2],  # 첫 번째 타임스텝에서의 예측 확률 분포
        [0.3, 0.2, 0.1, 0.4],  # 두 번째 타임스텝에서의 예측 확률 분포
        [0.25, 0.25, 0.25, 0.25],  # 세 번째 타임스텝에서의 예측 확률 분포
        [0.1, 0.2, 0.6, 0.1],  # 네 번째 타임스텝에서의 예측 확률 분포
        [0.05, 0.15, 0.7, 0.1]   # 다섯 번째 타임스텝(마지막)의 예측 확률 분포
    ]
]

predictions = np.array(predictions)

predicted_id = np.argmax(predictions[0, -1, :])
print("predicted_id = ", predicted_id)

"""
주어진 텍스트 text = "if you want you"에서 마지막 타임스텝은 RNN 모델이 입력으로 받는 시퀀스의 가장 마지막 문자를 가리킵니다.
이 경우, RNN은 주어진 입력 시퀀스에 대해 각 타임스텝에서 다음 문자를 예측합니다.

예시 설명
입력 시퀀스와 출력 시퀀스:

입력 시퀀스 (input_seq): "if you want yo" (마지막 문자는 제외)
출력 시퀀스 (output_seq): "f you want you" (첫 번째 문자는 제외)
타임스텝 구조:

입력 시퀀스는 문자의 인덱스로 변환되어 다음과 같은 형태가 됩니다:
입력 시퀀스:
input_seq → [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
(여기서 각 숫자는 문자 i, f, (공백), y, o, u, (공백), w, a, n, t, (공백), y, o에 해당)
마지막 타임스텝은 "o"에 해당합니다. 즉, 모델이 "if you want yo"를 입력으로 받고, 마지막 문자를 예측하는 것입니다.
모델의 예측:

모델은 마지막 입력 문자 "o"에 기반하여 다음 문자를 예측합니다. 이때, "o" 다음에 올 수 있는 문자 "u"를 예측하게 됩니다.
요약
주어진 텍스트 "if you want you"에서의 마지막 타임스텝은 "o"에 해당하며, 이는 input_seq의 가장 마지막 문자입니다.
모델은 이 마지막 문자를 기반으로 다음 문자를 예측하며, 예측 결과는 "u"가 될 것입니다.
"""

predicted_id =  2


'\n주어진 텍스트 text = "if you want you"에서 마지막 타임스텝은 RNN 모델이 입력으로 받는 시퀀스의 가장 마지막 문자를 가리킵니다. \n이 경우, RNN은 주어진 입력 시퀀스에 대해 각 타임스텝에서 다음 문자를 예측합니다.\n\n예시 설명\n입력 시퀀스와 출력 시퀀스:\n\n입력 시퀀스 (input_seq): "if you want yo" (마지막 문자는 제외)\n출력 시퀀스 (output_seq): "f you want you" (첫 번째 문자는 제외)\n타임스텝 구조:\n\n입력 시퀀스는 문자의 인덱스로 변환되어 다음과 같은 형태가 됩니다:\n입력 시퀀스:\ninput_seq → [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]\n(여기서 각 숫자는 문자 i, f, (공백), y, o, u, (공백), w, a, n, t, (공백), y, o에 해당)\n마지막 타임스텝은 "o"에 해당합니다. 즉, 모델이 "if you want yo"를 입력으로 받고, 마지막 문자를 예측하는 것입니다.\n모델의 예측:\n\n모델은 마지막 입력 문자 "o"에 기반하여 다음 문자를 예측합니다. 이때, "o" 다음에 올 수 있는 문자 "u"를 예측하게 됩니다.\n요약\n주어진 텍스트 "if you want you"에서의 마지막 타임스텝은 "o"에 해당하며, 이는 input_seq의 가장 마지막 문자입니다.\n모델은 이 마지막 문자를 기반으로 다음 문자를 예측하며, 예측 결과는 "u"가 될 것입니다.\n'